## 2021389_Lab Tasks


In [1]:
pip install torch torch-geometric

In [2]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data

In [3]:
# Define the Graph Convolutional Network (GCN) model
class GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)

        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

Creating a simple graph for training and testing

In [4]:
edge_index_train = torch.tensor([[0, 1, 1, 2, 2, 3, 4, 4, 5, 6, 6, 7, 8, 9],
                                 [1, 0, 2, 1, 3, 2, 4, 5, 4, 6, 8, 6, 9, 8]], dtype=torch.long)

In [5]:
x_train = torch.randn(10, 3)
y_train = torch.tensor([0, 1, 0, 1, 0, 1, 0, 1, 0, 1])

In [6]:
edge_index_test = torch.tensor([[0, 1, 1, 2, 2, 3, 3, 4, 4],
                                [1, 0, 2, 1, 3, 2, 4, 3, 0]], dtype=torch.long)  # Adjusted for 5 nodes
x_test = torch.randn(5, 3)

In [7]:
# Create PyTorch Geometric Data objects
data_train = Data(x=x_train, edge_index=edge_index_train, y=y_train)
data_test = Data(x=x_test, edge_index=edge_index_test)

In [8]:
# Initialize the GCN model
model = GCN(in_channels=3, hidden_channels=16, out_channels=2)

In [9]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [10]:
# Model Training
model.train()
optimizer.zero_grad()
out = model(data_train)
loss = criterion(out, data_train.y)
loss.backward()
optimizer.step()

In [11]:
# Model Testing
model.eval()
with torch.no_grad():
    pred = model(data_test)

In [12]:
# The 'pred' tensor now contains the predicted labels for the test nodes.
print("Predictions for test nodes:", pred.argmax(dim=1))

Predictions for test nodes: tensor([1, 1, 1, 0, 0])
